In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import pandas as pd
import os
from datasets import Dataset
from sklearn.model_selection import KFold

In [4]:
!git clone https://github.com/Marcos-Med/IC.git

Cloning into 'IC'...
remote: Enumerating objects: 21645, done.
remote: Counting objects: 100% (21645/21645), done.
remote: Compressing objects: 100% (14955/14955), done.
remote: Total 21645 (delta 6694), reused 21624 (delta 6683), pack-reused 0 (from 0)
Receiving objects: 100% (21645/21645), 21.58 MiB | 12.63 MiB/s, done.
Resolving deltas: 100% (6694/6694), done.
Updating files: 100% (21606/21606), done.


In [5]:
def load_corpus_fakeBR():
    base_path = "IC/corpura/fakebr/size_normalized_texts"

    fake_news = []
    true_news = []
    for root, dirs, files in os.walk(base_path + "/fake"):
        for file in files:
            if file.endswith('.txt'):
                with open(os.path.join(root, file), 'r', encoding='utf-8') as f:
                    text = f.read()
                    fake_news.append([text, 0])

    for root, dirs, files in os.walk(base_path + "/true"):
        for file in files:
            if file.endswith('.txt'):
                with open(os.path.join(root, file), 'r', encoding='utf-8') as f:
                    text = f.read()
                    true_news.append([text, 1])


    all_news = fake_news + true_news
    return pd.DataFrame(all_news, columns=['text', 'label'])

In [6]:
fake_br = load_corpus_fakeBR()
fake_br

,text,label
0,Acredite se quiser! Deputado cria projeto para...,0
1,Delegado da PF se revolta com exposição no MAM...,0
2,Empresário diz À PF que obras no sítio de Atib...,0
3,"Filho de Teori: ""Se eles compram o silêncio, p...",0
4,Candidatura do apresentador Sílvio Santos é um...,0
...,...,...
7195,Vídeo de cão correndo atrás de carro em Bauru ...,1
7196,Moro determina que José Dirceu deixe a prisão ...,1
7197,Gosto quando dizem que sou caso de marketing: ...,1
7198,"Governo dos EUA libera 2,9 mil documentos secr...",1


In [7]:
text = fake_br['text']
labels = fake_br['label']

In [8]:
model_name = "neuralmind/bert-base-portuguese-cased"

In [9]:
tokenizer = BertTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

In [10]:
def tokenize(data):
  return tokenizer(data['text'], padding='max_length', truncation=True, max_length=512)

In [11]:
dataset = Dataset.from_pandas(fake_br)
tokenized_dataset = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/7200 [00:00<?, ? examples/s]

In [12]:
tokenized_dataset = tokenized_dataset.remove_columns(['text'])
tokenized_dataset.set_format("torch")

In [13]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [14]:
Kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [15]:
os.environ["WANDB_DISABLED"] = "true"

In [16]:
import numpy as np
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # Cálculo das métricas
    accuracy = accuracy_score(labels, predictions)
    precision = precision_score(labels, predictions, average='binary')  # Para classificação binária
    recall = recall_score(labels, predictions, average='binary')  # Para classificação binária
    f1 = f1_score(labels, predictions, average='binary')  # Para classificação binária
    macro_f1 = f1_score(labels, predictions, average='macro')

    # Retornar as métricas calculadas
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "macro_f1": macro_f1
    }

In [17]:
results = []

In [18]:
for train_index, test_index, in Kf.split(tokenized_dataset):
    train_dataset = tokenized_dataset.select(train_index.tolist())
    test_dataset = tokenized_dataset.select(test_index.tolist())

    model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2) #Classificação binária

    # Paramêtros para realizar Fine Tuning no BERTimbau
    training_args = TrainingArguments(
        output_dir="./results",
        eval_strategy="epoch",
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=3,
        weight_decay=0.01,
        logging_dir="./logs",
        save_strategy="epoch"
        )
    #Objeto de treino
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=compute_metrics
    )

    #Treina o modelo
    trainer.train()

    #Avaliação do modelo BERTimbau
    metrics = trainer.evaluate()
    results.append(metrics)

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Macro F1
1,0.158300,0.058600,0.985417,0.991585,0.979224,0.985366,0.985416
2,0.048800,0.079120,0.985417,0.976871,0.994460,0.985587,0.985415
3,0.006400,0.047396,0.990972,0.993046,0.988920,0.990978,0.990972


...

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Macro F1
1,0.158600,0.038395,0.993056,0.998630,0.987805,0.993188,0.993053
2,0.065500,0.047074,0.993056,0.991892,0.994580,0.993234,0.993051
3,0.003600,0.029044,0.994444,0.994580,0.994580,0.994580,0.994441


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Macro F1
1,0.145100,0.027791,0.991667,0.994452,0.988966,0.991701,0.991667
2,0.058000,0.063505,0.989583,0.979730,1.000000,0.989761,0.989580
3,0.001300,0.020361,0.995833,0.994498,0.997241,0.995868,0.995833


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Macro F1
1,0.155800,0.040924,0.992361,0.995787,0.988842,0.992302,0.992361
2,0.042700,0.020891,0.994444,0.994421,0.994421,0.994421,0.994444
3,0.008700,0.026387,0.995139,0.995810,0.994421,0.995115,0.995139


In [21]:
model.save_pretrained("save_model")
tokenizer.save_pretrained("save_tokenizer")

('save_tokenizer/tokenizer_config.json',
 'save_tokenizer/special_tokens_map.json',
 'save_tokenizer/vocab.txt',
 'save_tokenizer/added_tokens.json')

In [22]:
results

[{'eval_loss': 0.04739566519856453,
  'eval_accuracy': 0.9909722222222223,
  'eval_precision': 0.9930458970792768,
  'eval_recall': 0.9889196675900277,
  'eval_f1': 0.9909784871616932,
  'eval_macro_f1': 0.9909722178685464,
  'eval_runtime': 41.7832,
  'eval_samples_per_second': 34.464,
  'eval_steps_per_second': 4.308,
  'epoch': 3.0},
 {'eval_loss': 0.062391046434640884,
  'eval_accuracy': 0.9895833333333334,
  'eval_precision': 0.9843971631205674,
  'eval_recall': 0.994269340974212,
  'eval_f1': 0.9893086243763364,
  'eval_macro_f1': 0.9895764516600707,
  'eval_runtime': 42.2791,
  'eval_samples_per_second': 34.059,
  'eval_steps_per_second': 4.257,
  'epoch': 3.0},
 {'eval_loss': 0.0290438923984766,
  'eval_accuracy': 0.9944444444444445,
  'eval_precision': 0.994579945799458,
  'eval_recall': 0.994579945799458,
  'eval_f1': 0.994579945799458,
  'eval_macro_f1': 0.9944409700507262,
  'eval_runtime': 41.8238,
  'eval_samples_per_second': 34.43,
  'eval_steps_per_second': 4.304,
  'ep

In [23]:
mean_metrics = {
    "accuracy": np.mean([metrics["eval_accuracy"] for metrics in results]),
    "precision": np.mean([metrics["eval_precision"] for metrics in results]),
    "recall": np.mean([metrics["eval_recall"] for metrics in results]),
    "f1": np.mean([metrics["eval_f1"] for metrics in results]),
    "macro_f1": np.mean([metrics["eval_macro_f1"] for metrics in results])
}

In [25]:

# Exibir as métricas médias
print("Métricas médias após K-fold cross-validation:")
print(mean_metrics)

# Salvar em CSV
df_metrics = pd.DataFrame(results)
df_metrics.to_csv("metrics_kfold.csv", index=False)

df = pd.DataFrame([mean_metrics])
df.to_csv("mean_kfold.csv", index=False)

Métricas médias após K-fold cross-validation:
{'accuracy': 0.9931944444444444, 'precision': 0.9924661997182994, 'recall': 0.9938863066232324, 'f1': 0.9931699937978108, 'macro_f1': 0.9931922915113794}
